### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_data_header, males_keys, females_keys

H5_dic = read_data_header()
chunk_size = 512*30

males = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'


Data with 40 male and female speakers


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model configuration

In [4]:
learning_rate = 0.001
regularization = 1e-3
batch_size = 1
N = 256
max_pool = 256

config_model = {}
config_model["alpha"] = learning_rate
config_model["reg"] = regularization
config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size
config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["type"] = "pretraining"
config_model["smooth_size"] = 10

# Model creation

In [5]:
from models.adapt import Adapt

adapt_model = Adapt(config_model=config_model, pretraining=True, folder='pretraining')
print 'Total name :' 
print adapt_model.runID
adapt_model.tensorboard_init()

ID : AdaptiveNet-winter-disk-4000
[<tf.Variable 'conv/W:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'smooth/smoothing_filter:0' shape=(1, 10, 1, 1) dtype=float32_ref>]
Total name :
AdaptiveNet-winter-disk-4000-N=256--alpha=0.001--batch_size=1--chunk_size=15360--maxpool=256--reg=0.001--smooth_size=10--type=pretraining-


# Pretraining the model 

In [6]:
nb_iterations = 1000

#initialize the model
adapt_model.init()
X_in, X_mix, Ind = mixed_data.get_batch(batch_size)

for i in range(nb_iterations):
    c = adapt_model.train(X_mix, X_in, learning_rate, i)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        adapt_model.save(i)

Step # 0  loss= 2.30674
DAS model saved at iteration number  0  with cost =  2.30674
Step # 1  loss= 2.86041
Step # 2  loss= 3.36773
Step # 3  loss= 3.88218
Step # 4  loss= 4.30282
Step # 5  loss= 4.79605
Step # 6  loss= 5.23362
Step # 7  loss= 5.7408
Step # 8  loss= 6.12815
Step # 9  loss= 6.50571
Step # 10  loss= 7.00692
Step # 11  loss= 7.53687
Step # 12  loss= 8.03594
Step # 13  loss= 8.63175
Step # 14  loss= 9.2146
Step # 15  loss= 9.84159


KeyboardInterrupt: 